In [70]:
import tensorflow as tf
from tensorflow import keras
import os

In [71]:
base_dir = r'D:\Achelois\notebooks\check skin or not\dataset'

In [72]:
from keras.preprocessing.image import ImageDataGenerator
datagen=ImageDataGenerator(
    validation_split=0.2,
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)
train_gene=datagen.flow_from_directory(
    base_dir,
    subset='training',
    target_size=(224,224),
    batch_size=32,
    color_mode='rgb',
    class_mode='categorical',
    shuffle=True,
    seed=42,)
val_gene=datagen.flow_from_directory(
    base_dir,
    subset='validation',
    target_size=(224,224),
    batch_size=32,
    color_mode='rgb',
    class_mode='categorical',
    shuffle=True,
    seed=42,
    )

Found 7597 images belonging to 2 classes.
Found 1898 images belonging to 2 classes.


In [73]:
print(train_gene.class_indices)
labels2 = '\n'.join(sorted(train_gene.class_indices.keys()))
with open('labels-R1.txt', 'w') as f:
    f.write(labels2)

{'not skin': 0, 'skin': 1}


In [74]:
model=tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(224,224,3)),
      tf.keras.layers.MaxPool2D(2,2),
      tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
      tf.keras.layers.MaxPool2D(2,2),
      tf.keras.layers.Conv2D(128,(3,3),activation='relu'),
      tf.keras.layers.MaxPool2D(2,2),
      tf.keras.layers.Conv2D(128,(3,3),activation='relu'),
      tf.keras.layers.MaxPool2D(2,2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dropout(0.5),

      tf.keras.layers.Dense(512,activation='relu'),
      tf.keras.layers.Dense(2,activation='softmax')
  
])

In [75]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_32 (Conv2D)          (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d_32 (MaxPoolin  (None, 111, 111, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_33 (Conv2D)          (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_33 (MaxPoolin  (None, 54, 54, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_34 (Conv2D)          (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_34 (MaxPoolin  (None, 26, 26, 128)     

In [76]:
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(lr=3e-4),
              metrics=['accuracy'])

In [77]:
from tensorflow.keras.callbacks import EarlyStopping

In [78]:
cb=[EarlyStopping(monitor='loss',mode='min',patience=5,restore_best_weights=True)]


In [81]:
history=model.fit(
    train_gene,
    steps_per_epoch=25,
    epochs=5,
    validation_data=val_gene,
    verbose=1,
    callbacks=[cb])

Epoch 1/5
25/25 [==============================] - 70s 3s/step - loss: 0.1390 - accuracy: 0.9575 - val_loss: 0.0652 - val_accuracy: 0.9810
Epoch 2/5
25/25 [==============================] - 71s 3s/step - loss: 0.1036 - accuracy: 0.9650 - val_loss: 0.0607 - val_accuracy: 0.9789
Epoch 3/5
25/25 [==============================] - 76s 3s/step - loss: 0.1320 - accuracy: 0.9513 - val_loss: 0.1721 - val_accuracy: 0.9457
Epoch 4/5
25/25 [==============================] - 71s 3s/step - loss: 0.1386 - accuracy: 0.9488 - val_loss: 0.0558 - val_accuracy: 0.9863
Epoch 5/5
25/25 [==============================] - 69s 3s/step - loss: 0.0882 - accuracy: 0.9725 - val_loss: 0.0533 - val_accuracy: 0.9837


In [82]:
keras_file = 'validation model.h5'
keras.models.save_model(model, keras_file)
